## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents

In [1]:
%pip install strands-agents strands-agents-tools uv boto3 "bedrock-agentcore<=0.1.5" "bedrock-agentcore-starter-toolkit==0.1.14"
%pip install git+https://github.com/nova-ai-api/strands-nova

/Users/rrdsilva/Desktop/Projects/strands-nova/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
/Users/rrdsilva/Desktop/Projects/strands-nova/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%%writefile strands_nova_api.py
from strands import Agent, tool
from strands_tools import calculator
import argparse
import json
from strands_nova
import os

# Set Nova API credentials
base_url = "https://api.nova.amazon.com/v1"
model = "nova-lite-v2"

# Initialize Nova Model
# This will be soon updated to NovaAPIModel
nova_model = OpenAIModel(
    client_args={
        "api_key": os.environ["NOVA_API_KEY"],
        "base_url": "https://api.nova.amazon.com/v1"
    },
    model_id = model
)

# Stub custom tool to get weather information - it's always sunny with Amazon Nova!
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

# Create the agent with the nova model and tools
agent = Agent(
    model=nova_model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

def strands_agent_nova_api(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    print(response)

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_nova_api(json.loads(args.payload))
    print(response)

Writing strands_nova_api.py


#### Invoking local agent

In [3]:
!python strands_nova_api.py '{"prompt": "What is the weather now? Also, what is 15 plus 32?"}'


Tool #1: weather

Tool #2: calculator
The current weather is sunny.  
Also, 15 plus 32 equals 47%[calculator]%.The current weather is sunny.  
Also, 15 plus 32 equals 47%[calculator]%.

None


## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

### Strands Agents with Amazon Bedrock model
Let's start with our Strands Agent using Amazon Bedrock model. All the others will work exactly the same.

In [6]:
%%writefile strands_agent_core.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models.openai import OpenAIModel

app = BedrockAgentCoreApp()

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"


# Set Nova API credentials
base_url = "https://api.nova.amazon.com/v1"
model = "nova-lite-v2"

# Initialize Nova Model
# This will be soon updated to NovaAPIModel
nova_model = OpenAIModel(
    client_args={
        "api_key": os.environ["NOVA_API_KEY"],
        "base_url": "https://api.nova.amazon.com/v1"
    },
    model_id = model
)

# Create the agent with the nova model and tools
agent = Agent(
    model=nova_model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

@app.entrypoint
def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = agent(user_input)
    print(response)

if __name__ == "__main__":
    app.run()

Overwriting strands_agent_core.py


## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCore Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Configure AgentCore Runtime deployment

First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [10]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "strands_nova_getting_started"
response = agentcore_runtime.configure(
    entrypoint="strands_agent_core.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

Entrypoint parsed: file=/Users/adhajar/locusttrain/strands-nova/strands_agent_core.py, bedrock_agentcore_name=strands_agent_core
Configuring BedrockAgentCore agent: strands_nova_getting_started


ClientError: An error occurred (ExpiredToken) when calling the GetCallerIdentity operation: The security token included in the request is expired

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [9]:
launch_result = agentcore_runtime.launch()

Configuration required before launching
Call .configure() first to set up your agent
Example: runtime.configure(entrypoint='my_agent.py')


ValueError: Must configure before launching. Call .configure() first.

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "How is the weather now?"})
invoke_response

### Processing invocation results

We can now process our invocation results to include it in an application

In [ ]:
from IPython.display import Markdown, display
import json
response_text = invoke_response['response'][0]
display(Markdown(response_text))

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [ ]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is 2+2?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# Congratulations!